In [2]:
import numpy as np
import seaborn as sns 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
df = pd.read_csv("../data/cardio_train_clean.csv", sep = ";", index_col="id")
df.info()
df.shape
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 68562 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          68562 non-null  float64
 1   gender       68562 non-null  int64  
 2   height       68562 non-null  int64  
 3   weight       68562 non-null  float64
 4   ap_hi        68562 non-null  int64  
 5   ap_lo        68562 non-null  int64  
 6   cholesterol  68562 non-null  int64  
 7   gluc         68562 non-null  int64  
 8   smoke        68562 non-null  int64  
 9   alco         68562 non-null  int64  
 10  active       68562 non-null  int64  
 11  cardio       68562 non-null  int64  
dtypes: float64(2), int64(10)
memory usage: 6.8 MB


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,50.391781,2,168,62.0,110,80,1,1,0,0,1,0
1,55.419178,1,156,85.0,140,90,3,1,0,0,1,1
2,51.663014,1,165,64.0,130,70,3,1,0,0,0,1
3,48.282192,2,169,82.0,150,100,1,1,0,0,1,1
4,47.873973,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
# Stocker les index des positifs (smoke ou alco)
positive_idx = df[(df['smoke'] == 1) | (df['alco'] == 1)].index

---

### Activité physique

|Variable|interpretation| valeur élevé =|
|:-|:-|:-|
| CHOLESTEROL | 1 : normal<br> 2 : supérieur à la normale<br> 3 : largement supérieur à la normale| incidence négative|
| GLUCOSE     | 1 : normal<br> 2 : supérieur à la normale<br> 3 : largement supérieur à la normale| incidence négative|
| SMOKE       | 0 : non fumeur<br>1 : fumeur| incidence négative|
| ALCOHOL     | 0 : non consomateur<br> 1 : consomateur| incidence négative|
| PHYSICAL_ACTIVITY | 0 : non, 1 : oui| incidence positive|



Pour l'activité physique, la logique est inversée<br>
Nous alons donc intervertir les 0 et 1 de `active` pour respecter la même logique que sur les autre variables.

In [5]:
# Inversionde 1 et 0 de la variable 'active'
df['active'] = 1 - df['active']

----
## gender
2 = homme<br>
1 = femme<br>
Mais il s'agit d'une donnée Cardinale, pas ordinale><br>
Nous utilisons OneHotEncoder pour remplacer `gender` par `male`et `female`.

In [6]:
encoder = OneHotEncoder(sparse_output=False)

# Encode la colonne 'gender'
gender_encoded = encoder.fit_transform(df[['gender']])

# Corriger l’accès aux catégories
categories = encoder.categories_[0]  # Extraire la seule liste
column_names = ['female' if val == 1 else 'male' for val in categories]

# Créer un DataFrame avec les nouvelles colonnes
gender_df = pd.DataFrame(gender_encoded, columns=column_names, index=df.index)

# Remplacer la colonne d’origine
df = pd.concat([df.drop(columns='gender'), gender_df], axis=1)

---
## IMC

### Indice de Masse Corporelle

Calcul de l'IMC :

$$
IMC = \frac{\text{poids (kg)}}{\left( \frac{\text{taille (cm)}}{100} \right)^2}
$$

​
Catégorisation médicale simplifiée :

|IMC	|Catégorie|	Valeur|
|-|-|-|
|< 25	|Normal	|1|
|25 ≤ IMC < 30	|Surpoids |	2|
|≥ 30	|Obésité |	3|

In [7]:
# Calcul de l'IMC
df['imc'] = df['weight'] / ((df['height'] / 100) ** 2)

# Création de la variable catégorielle
def categoriser_imc(imc):
    if imc < 25:
        return 1  # Normal
    elif imc < 30:
        return 2  # Supérieur à la normale
    else:
        return 3  # Très supérieur à la normale

df['imc'] = df['imc'].apply(categoriser_imc)


---
## pressure

### Tension artérielle

| Diastolique (`ap_lo`) | Systolique (`ap_hi`) | Interprétation        | Code |
| -------------------- | --------------------- | --------------------- | ---- |
| < 80                | < 120                | Normale               | 1    |
| 80–89             | 120–139               | Élevée (à surveiller) | 2    |
| ≥ 90                | ≥ 140                  | Hypertension          | 3    |


Si la systolique ou la diastolique dépasse un seuil, on prend la catégorie la plus élevée.

In [8]:
def classifier_pressure(row):

    if row['ap_hi'] < 120 and row['ap_lo']< 80:
        return 1  # Normal
    elif row['ap_hi'] < 140 and row['ap_lo'] < 90:
        return 2  # A surveiller
    else:
        return 3  # Hypertension

df['pressure'] = df.apply(classifier_pressure, axis=1)


## Autres variables créées

In [9]:
df['risk_behavior'] = df['smoke'] + df['alco']
df['risk_behavior'].value_counts()

risk_behavior
0    60658
1     6090
2     1814
Name: count, dtype: int64

In [10]:
df['age_risk'] =  df['age'] * df['risk_behavior']
df['age_risk'].value_counts()

age_risk
0.000000      60658
49.764384         7
56.104110         6
55.975342         5
55.391781         5
              ...  
57.961644         1
117.660274        1
79.446575         1
56.512329         1
49.665753         1
Name: count, Length: 5469, dtype: int64

In [11]:
df['imc_pressure'] =  df['imc'] * df['pressure']
df['imc_pressure'].value_counts()

imc_pressure
2    18315
6    16042
4    12885
9     9091
3     7229
1     5000
Name: count, dtype: int64

In [12]:
df['chole_gluc'] =  df['cholesterol'] * df['gluc']
df['chole_gluc'].value_counts()

chole_gluc
1    47760
2     8742
3     5435
9     3389
4     2365
6      871
Name: count, dtype: int64

---
## Export du csv optimisé

In [13]:
# df.to_csv('../data/cardio_optimized_2.csv', sep = ";")

---
### csv optimisé et allegé

suppression de :<br>
height, weight (remplacé par IMC)<br>
ap_lo, ap_hi (remplacé par pressure)


In [14]:
df = df.drop(columns=['height', 'weight', 'ap_lo', 'ap_hi', 'cholesterol', 'gluc', 'alco', 'smoke'])


### Export du csv optimisé et allegé


In [15]:
# df.to_csv('../data/cardio_light_2.csv', sep = ";")

---
### df_scaled
#### Standardisation de age et age_risk

In [16]:
# non_zero = df['age_risk'] != 0
columns_to_scale = ['age','age_risk']
scaler = StandardScaler()

df_scaled = df.copy()
df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])
# df_scaled.loc[non_zero, 'age_risk'] = scaler.fit_transform(df.loc[non_zero, ['age_risk']])


In [17]:
df_scaled['age_risk'].value_counts()

age_risk
-0.337109    60658
 1.923118        7
 2.211059        6
 2.205211        5
 2.178706        5
             ...  
 2.295426        1
 5.006851        1
 3.271240        1
 2.229600        1
 1.918638        1
Name: count, Length: 5469, dtype: int64

### Export du df standardisé

In [18]:
# df.to_csv('../data/cardio_light_std.csv', sep = ";")
# df_scaled.to_csv('../data/cardio_prepro_light_std.csv', sep = ";")

---
### df_scaled_light
#### Deuxième supression de variables

In [19]:
# Supression des variable redondantes
df_scaled_light = df_scaled.drop(columns=['imc', 'pressure','age_risk'])
df_scaled_light.head()

,age,active,cardio,female,male,risk_behavior,imc_pressure,chole_gluc
id,,,,,,,,
0,-0.434097,0,0,0.0,1.0,0,2,1
1,0.309546,0,1,1.0,0.0,0,9,3
2,-0.246058,1,1,1.0,0.0,0,2,3
3,-0.746143,0,1,0.0,1.0,0,6,1
4,-0.806526,1,0,1.0,0.0,0,1,1


---
## equilibrage de smoke 0/1 et alco 0/1
- comptage des observations qui ont smoke ou alco positive
- **Undersampling** limitation à ce même nombre des observations qui ont smoke ou alco négative.

**Objectif :** équilibrer les classes en training, tout en gardant trace des id d’origine pour pouvoir bien retrouver facilement les observations dans le test.

In [ ]:
# # 1. Extraire les positifs selon les index enregistrés
# positifs = df_scaled_light.loc[positive_idx]

# # 2. Définir les négatifs (les autres)
# negatifs = df_scaled_light.drop(index=positive_idx)

# # 3. Échantillonnage aléatoire de négatifs pour équilibrer
# negatifs_sample = negatifs.sample(n=len(positifs), random_state=42)

# # 4. Concaténer les deux groupes et mélanger
# df_balanced = pd.concat([positifs, negatifs_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# df_scaled_light = DataFrame complet
# index           = 'id'
# cible           = 'cardio' 

# Index des positifs (classe minoritaire)
positive_idx = df_scaled_light[df_scaled_light['cardio'] == 1].index

# Extrait les positifs (training minoritaire)
positifs = df_scaled_light.loc[positive_idx]

# 4. Extraire les négatifs
negatifs = df_scaled_light[df_scaled_light['cardio'] == 0]

# Échantillonnage aléatoire des négatifs pour équilibrer
negatifs_sample = negatifs.sample(n=len(positifs), random_state=42)

# Concaténe positifs + négatifs échantillonnés, puis mélange
df_balanced = pd.concat([positifs, negatifs_sample]).sample(frac=1, random_state=42)

In [27]:
df_balanced['cardio'].value_counts()

cardio
0    33934
1    33934
Name: count, dtype: int64

In [21]:

# Vérifie que l’index est bien toujours 'id' :
print(df_balanced.index.name)  # doit afficher 'id'

id


In [ ]:
print(df_balanced.index.is_unique)  # doit être True

True


## Export de l’échantillon équilibré - Train

In [23]:
df_balanced.to_csv('../data/cardio_balanced_train.csv', sep=';', index=True)

## Export de l’échantillon équilibré - Test

In [24]:

# 9. Pour le test, tu récupères les ids qui n’ont pas été pris dans l’échantillonnage
test_idx = df_scaled_light.index.difference(df_balanced.index)
df_test = df_scaled_light.loc[test_idx]

# 10. Export ou utilisation du jeu de test avec les id conservés
df_test.to_csv('../data/cardio_balanced_test.csv', sep=';', index=True)


In [25]:
df_balanced.head()

,age,active,cardio,female,male,risk_behavior,imc_pressure,chole_gluc
id,,,,,,,,
56633,-0.461654,0,0,1.0,0.0,0,1,1
81907,-0.217691,1,1,0.0,1.0,0,3,1
7463,0.075714,0,0,1.0,0.0,0,6,1
55907,-0.743711,0,1,0.0,1.0,0,2,1
54568,1.327545,0,1,1.0,0.0,0,9,1
